## Initialization
Importing required libraries and reading in all phone reviews CSV files.

In [1]:
import pandas as pd
import numpy as np
import json, re, os
from datetime import datetime, timedelta
#from extractPhoneModels import getEntities
from textblob import TextBlob
from multiprocessing import Pool

phone_df1 = pd.read_csv("phone_models_50.csv")
phone_df2 = pd.read_csv("phone_models_51to100.csv")
phone_df3 = pd.read_csv("phone_models_101to150.csv")
phone_df4 = pd.read_csv("phone_models_151to200.csv")
phone_df5 = pd.read_csv("phone_models_201to250.csv")
phone_df6 = pd.read_csv("phone_models_251to350.csv")
phone_df7 = pd.read_csv("phone_models_351to450.csv")
phone_df8 = pd.read_csv("phone_models_451to550.csv")
phone_df9 = pd.read_csv("phone_models_551to650.csv")
phone_df10 = pd.read_csv("phone_models_651to750.csv")
phone_df11 = pd.read_csv("phone_models_751to850.csv")
all_phones_df = phone_df1.append([phone_df2, phone_df3, phone_df4, phone_df5, phone_df6, phone_df7, phone_df8, phone_df9, phone_df10, phone_df11])
all_phones_df = all_phones_df.reset_index()
all_phones_df.drop('index', axis=1, inplace=True)
all_phones_df.tail()

,phone_brand,phone_model,phone_reviews,phone_reviews_dates
845,Micromax,Selfie 2 Q4311,;;;;;Getting automatic screen lock issue. Whil...,"[datetime.date(2018, 2, 2), datetime.date(2017..."
846,Micromax,Canvas 1,;;;;;I'm facing hanging problem whenever I on ...,"[datetime.date(2018, 3, 16), datetime.date(201..."
847,Micromax,Canvas 2 Q4310,;;;;;Wrost phone experience ever . heating pr...,"[datetime.date(2018, 3, 26), datetime.date(201..."
848,Micromax,Canvas Evok Power Q4260,;;;;;My phone is not up to date so plz send me...,"[datetime.date(2018, 1, 25), datetime.date(201..."
849,Micromax,Canvas Evok Note E453,;;;;;Buy other brand's phone. At this price yo...,"[datetime.date(2017, 11, 28), datetime.date(20..."


In [2]:
# extractPhoneModels.py functions
brands = ["LG", "Google", "Sony", "Samsung", "Xiaomi", "Moto", "Nokia", "BlackBerry", "ZTE", "Honor", "Razer", "Asus", "Elephone", "Huawei", "Maze", "Nextbit", "HTC", "OnePlus", "Nexus", "BLU"]
allModelTokens = ["3s", "", "ips", "828", "a8", "zb452kg", "735", "6c", "n8", "blu", "3c", "q415", "fusion", "mini", "cube", "s5020", "f1s", "r7", "xa1", "500", "zc521tl", "smart", "sl6010", "s9+", "ze520kl", "dash", "lt2000", "320", "y3", "b", "xperia", "z3", "q2100", "5x", "ipaq", "526", "s5527", "q301", "9780", "luge", "v30", "m5", "yunicorn", "verykool", "eluga", "astro", "635", "zb450kl", "x4+", "s4008", "s5.1", "106", "x50", "pegasus", "ar", "tg02", "a7", "l2", "ts605", "steel", "150", "105", "mi", "a1", "lifestyle", "s500e", "hardcase", "9810", "2s", "voice", "surface", "q426", "s6004", "u11", "life", "sl5560", "zero", "s550", "pro3", "5t", "p55", "e5", "cdma", "q900s", "y25", "1x", "sl4502", "q385", "stylus", "4g", "enjoy", "xcover", "x41", "dual", "k7", "zc554kl", "4-core", "asus", "z5", "a55", "v3max", "i5", "a48", "mix", "s5019", "might", "k01", "8", "zc520kl", "ts608", "y69", "x20", "q413", "ray", "a", "black", "1", "tx62", "q5", "y67", "star", "p7+", "207", "q500s", "zc551kl", "p'9983", "s4007", "535", "ai", "4+", "harmony", "coach", "s5031", "hw6910", "ze552kl", "330g", "tank", "y7", "slate6", "3310", "one", "phantom", "speed", "a550s", "x5", "s5001", "phablet", "s4009", "9320", "z200", "zb551kl", "4k", "mega", "magic", "onem", "selfie", "v20", "mirror", "platinum", "a1000s", "xolo", "a97", "z330", "zc520tl", "g450", "s5028", "p1", "play", "v3", "71", "3x", "s10c", "a53", "supreme", "s6005", "prime", "mx-200tv", "s11", "spear", "glisten", "me", "v7", "q1000s", "7", "cyprus", "sl5050", "s5034", "p9", "e10", "a50", "a76+", "p100", "plum", "first", "e483", "9330", "orbit", "435", "tribute", "zc550kl", "s10b", "y6", "400", "a4", "x10", "yutopia", "nexus", "a1010", "plus", "note", "y37", "z10", "z25", "a31", "515", "ram", "y35", "626s", "cosmo", "216", "huawei", "s10", "s11s", "630", "r2", "3g", "r1x", "mx-110", "6a", "g4", "infinity", "s5030", "5s", "q700s", "e3", "mx6", "m2", "2gb+", "s5528", "m3", "m330", "motorola", "c5", "grand", "dtek60", "fe", "phab2", "q402", "x6", "a59", "s5525", "q8", "ts705", "ts32", "3gb", "go", "s6", "j3", "zuk", "a9s", "a71", "1020", "neo", "spark", "e9s", "a39", "k5", "yu", "x5pro", "9360", "k320t", "z80", "p", "sl5029", "mx5", "lumia", "z4v", "9800", "m9", "vodafone", "m6", "8110", "ultra", "redmi", "z30", "z500", "twom", "deluxe", "a79", "x19", "z50", "g389f", "sl5011", "helix", "gravity", "200", "q4201", "curve", "9930", "q350", "gator", "energizer", "p550s", "nova", "3v", "9", "yunique", "z60", "m5s", "p'9981", "h6320", "maverick", "s5200", "a33", "a77", "1520", "sim", "ts803", "550", "s5025", "610c", "sync", "910c", "a6", "microsoft", "s6005x", "x55", "i2", "z11", "sl5550", "4", "a8+", "3000", "jade", "9720", "porsche", "s", "xa2", "lava", "led", "c2", "a2", "ts921", "rocket", "q352", "x", "v5s", "(3.5)", "icon", "503", "blade", "c10", "mx2", "2520", "canvas", "638", "meizu", "4.0b", "cam", "metal", "stylo", "yuphoria", "x6s", "activ", "q520s", "x7", "era", "q300", "f2", "trigger", "club", "camera", "8x-1020", "mx-210tv", "g5s", "q700", "galaxy", "525", "240", "i4", "6s", "q1000", "a89", "pilot", "e20", "xiaomi", "55", "4c", "emerge", "boot", "lg", "x46", "lemon", "x1", "evo", "le", "e8", "330", "iris", "view", "r7s", "130", "z630s", "yureka", "pro", "u11+", "data", "power", "xzs", "desire", "maven", "zb501kl", "q510s", "virtue", "dtek50", "zd552kl", "leap", "g6", "8x-1000", "pixel", "p8", "passport", "a3", "q4310", "duo", "zd553kl", "oppo", "m1", "on8", "s8", "p2", "4a", "p88", "active", "l1", "asha", "premium", "e", "zenfone", "skin", "ze553kl", "9300", "priv", "i9", "208", "samsung", "950", "q610s", "chat", "a37", "hawkeye", "energy", "e520", "a76", "z530s", "j5", "a9", "a82", "xa", "x28", "leeco", "g3", "z4", "xl", "x9", "q4311", "maxwest", "zest", "100", "compass", "honor", "hw6515", "shine", "mx4", "a67", "v5", "u", "x17", "hive", "giant", "k20", "f5", "nokia", "2x", "a73", "oneplus", "9860", "aristo", "532", "v520kl", "p600s", "y5", "zs570kl", "rw6818", "laser", "h6325", "222", "430", "touch", "4.0", "a68", "omega", "jr", "z6", "micromax", "j1", "marathon", "g910", "screen", "x4", "alcatel", "moto", "lotus", "q440", "pixi", "elife", "730", "edition", "y15s", "k6", "fierce", "u10", "motion", "a5", "z220", "zc553kl", "r9", "ts808", "g710", "r9s", "iii", "m3e", "530", "butterfly", "f1", "is12t", "advance", "joy", "904t", "z520", "520", "x1s", "corporate", "ud", "u20", "5a", "z320", "panasonic", "compact", "mx5e", "tab", "y31", "jr.", "m9+", "classic", "9900", "turbo", "studio", "-", "keyone", "flash", "x5max+", "wave", "g500", "s5524", "style", "326g", "n2", "c9", "650", "y1", "tg01", "nubia", "tempo", "g", "10", "z90", "y51", "700", "7s", "q462", "aurora", "s7", "pop", "vita", "930", "r11", "se", "hp", "s9", "xz1", "5c", "fuel", "live", "6200t", "iphone", "blackberry", "n1", "625", "two", "5400t", "z17s", "55m", "705t", "x81", "evok", "4600", "v10", "power2", "a57", "vivo", "628", "p5w", "zs551kl", "sony", "107", "f3", "leo", "v8", "r5s", "voicetab", "1s", "q428", "acer", "pre", "iv", "514", "hd", "9370", "e600", "4s", "x5max", "maxx", "zte", "xplay6", "vdeo", "on5", "force", "a6600", "vista", "google", "m2017", "torch", "s5530", "amaze", "8x-1100", "m9s", "zu680kl", "7x", "c", "note8", "z630", "e453", "m220", "4400", "bharat", "y53", "q460", "tx80", "(5.5)", "220", "z17", "e2", "y55s", "z", "a601", "venture", "xp", "y6ii", "mx3", "lte", "g810", "z530", "j", "q1020", "a88", "non", "ts10", "a700s", "fire", "k10", "820g+", "zb552kl", "m3s", "q424", "slate7", "830", "5.5", "9380", "orion", "elite", "k8", "w909", "h550s", "l5", "s5029", "eclipse", "droid", "rw6815", "axon", "r11s", "g5", "850", "veer", "9220", "q437", "xtreme", "bold", "rw6828", "max", "vibe", "novo", "on7", "3", "626", "zb500kl", "win", "a44", "4.5", "hero", "e4816", "q381", "messenger", "q6", "l3", "s6s", "android", "g900", "v2", "s5035", "a520", "zb553kl", "mate", "pioneer", "wine", "x38", "h6315", "lite", "5400", "9850", "s5027", "nitro", "j2", "gionee", "e4815", "p'9982", "pulsemix", "s5021", "bello", "warp", "x11", "9350", "xplay5", "903t", "k4", "sirocco", "htc", "x6plus", "crystal", "z2", "z410", "pulse", "ace", "windows", "655", "f103", "p85", "zs571kl", "a7000", "minis", "m", "x2", "xl2", "p5", "s3", "c7", "e4", "u5", "l4", "m5c", "performance", "825", "q10", "hw6915", "s5526", "v7+", "s5205", "nxt", "v1", "luna", "s8+", "j8m", "728", "502", "8x-1200", "2+", "liquid", "9790", "v", "p10", "bolt", "s5007", "p99", "express", "5", "x+", "mark", "vice", "k3", "m320", "6", "q710s", "i3", "230", "sl5565", "225", "apple", "hw6510", "5m", "xz", "9670", "j7", "s5518q", "kin", "e700", "g920", "a51", "3t", "zoom", "idol", "640", "mach", "axe", "design", "a83", "a32", "x3", "1030", "sl5200", "edge", "ii", "4x", "540", "mx", "primo", "phone", "x9s", "1320", "toshiba", "108", "youth", "h240s", "s1", "opus", "v9", "/", "p7", "q4601", "check", "m6s", "eyes", "lenovo", "zs550kl", "3.5", "r1", "note7", "215", "2", "q4260", "6200", "alpha", "j8", "1330", "e457", "m3x", "ze554kl", "s4513", "m7", "p91", "5.0", "a72"]
fragmentLookup = {"moto g": "Moto G", "samsung galaxy s8+": "Samsung Galaxy S8+", "samsung galaxy s7 edge": "Samsung Galaxy S7 Edge", "lg v30s": "LG V30S", "xperia x performance": "Sony Xperia X Performance", "note 4": "Samsung Galaxy Note 4", "nokia 7 plus": "Nokia 7 Plus", "zenfone": "Asus Zenfone 5Z", "galaxy a8+": "Samsung Galaxy A8+", "huawei mediapad": "Huawei MediaPad M5", "asus zenfone": "Asus Zenfone 5Z", "lg x cam": "LG X cam", "7x": "Honor 7X", "oneplus 2": "OnePlus 2", "lg x screen": "LG X screen", "huawei mate 8": "Huawei Mate 8", "huawei p10": "Huawei P10", "lg g5": "LG G5", "vowney": "Elephone Vowney", "s7 edge": "Samsung Galaxy S7 Edge", "nexus 9": "Nexus 9", "google nexus 5": "Google Nexus 5", "huawei mediapad m5": "Huawei MediaPad M5", "xa": "Sony Xperia XA", "moto x": "Moto X", "nokia 7": "Nokia 7", "6p": "Huawei Nexus 6P", "htc u11+": "HTC U11+", "xl": "Google Pixel 2 XL", "s9+": "Samsung Galaxy S9+", "samsung galaxy s9+": "Samsung Galaxy S9+", "honor view 10": "Honor View 10", "nexus 6p": "Huawei Nexus 6P", "moto x style": "Moto X Style", "g 2015": "Moto G 2015", "google nexus": "Google Nexus 5", "8l": "BLU VIVO 8L", "zte axon 7": "ZTE Axon 7", "z3": "Sony Xperia Z3 Compact", "htc one": "HTC One", "zenfone 5z": "Asus Zenfone 5Z", "m5": "Huawei MediaPad M5", "htc desire": "HTC Desire 825", "axon 7": "ZTE Axon 7", "s6": "Samsung Galaxy S6", "moto g4 plus": "Moto G4 Plus", "google pixel": "Google Pixel", "galaxy s9+": "Samsung Galaxy S9+", "honor 9i": "Honor 9i", "xperia z5": "Sony Xperia Z5", "g5": "LG G5", "blu vivo": "BLU VIVO 8L", "a1": "Xiaomi Mi A1", "x cam": "LG X cam", "oneplus 3": "OnePlus 3", "elephone s8": "Elephone S8", "zte axon m": "ZTE Axon M", "nextbit robin": "Nextbit Robin", "htc 10": "HTC 10", "axon m": "ZTE Axon M", "galaxy s7": "Samsung Galaxy S7", "samsung galaxy s7": "Samsung Galaxy S7", "samsung galaxy s8": "Samsung Galaxy S8", "5t": "OnePlus 5T", "samsung galaxy s 5": "Samsung Galaxy S 5", "google pixel 2 xl": "Google Pixel 2 XL", "7 plus": "Nokia 7 Plus", "zte blade": "ZTE Blade V9", "oneplus x": "OnePlus X", "x performance": "Sony Xperia X Performance", "moto g 2015": "Moto G 2015", "sony xperia z3": "Sony Xperia Z3 Compact", "moto g4": "Moto G4 Plus", "z3 compact": "Sony Xperia Z3 Compact", "mediapad m5": "Huawei MediaPad M5", "mediapad": "Huawei MediaPad M5", "lg g6": "LG G6", "razer phone": "Razer Phone", "xperia z3 compact": "Sony Xperia Z3 Compact", "xiaomi mi 5": "Xiaomi Mi 5", "galaxy s6": "Samsung Galaxy S6", "9i": "Honor 9i", "huawei mate 10": "Huawei Mate 10", "z5": "Sony Xperia Z5", "galaxy s9": "Samsung Galaxy S9", "oneplus 5": "OnePlus 5", "htc u11 eyes": "HTC U11 Eyes", "oneplus 5t": "OnePlus 5T", "nexus 5": "Google Nexus 5", "samsung galaxy note 4": "Samsung Galaxy Note 4", "x style": "Moto X Style", "blackberry motion": "BlackBerry Motion", "xperia xzs": "Sony Xperia XZs", "moto e 2015": "Moto E 2015", "galaxy note 4": "Samsung Galaxy Note 4", "5z": "Asus Zenfone 5Z", "a8+": "Samsung Galaxy A8+", "x screen": "LG X screen", "galaxy s 5": "Samsung Galaxy S 5", "blade v9": "ZTE Blade V9", "blu vivo 8l": "BLU VIVO 8L", "compact": "Sony Xperia Z3 Compact", "desire 825": "HTC Desire 825", "samsung galaxy s9": "Samsung Galaxy S9", "galaxy s8+": "Samsung Galaxy S8+", "e 2015": "Moto E 2015", "sony xperia z5": "Sony Xperia Z5", "v30s": "LG V30S", "blade": "ZTE Blade V9", "sony xperia x performance": "Sony Xperia X Performance", "htc u11 life": "HTC U11 Life", "s8+": "Samsung Galaxy S8+", "sony xperia z3 compact": "Sony Xperia Z3 Compact", "oneplus one": "OnePlus One", "elephone vowney": "Elephone Vowney", "htc desire 825": "HTC Desire 825", "u11 eyes": "HTC U11 Eyes", "pixel 2": "Google Pixel 2", "s7": "Samsung Galaxy S7", "mi 5": "Xiaomi Mi 5", "xperia z3": "Sony Xperia Z3 Compact", "mate 10": "Huawei Mate 10", "p10": "Huawei P10", "pixel 2 xl": "Google Pixel 2 XL", "moto e": "Moto E 2015", "9 lite": "Honor 9 Lite", "pixel c": "Google Pixel C", "s9": "Samsung Galaxy S9", "huawei nexus": "Huawei Nexus 6P", "google pixel c": "Google Pixel C", "mate 8": "Huawei Mate 8", "lg v10": "LG V10", "v20": "LG V20", "xperia xa": "Sony Xperia XA", "vivo 8l": "BLU VIVO 8L", "sony xperia xa": "Sony Xperia XA", "robin": "Nextbit Robin", "asus zenfone 5z": "Asus Zenfone 5Z", "zte blade v9": "ZTE Blade V9", "moto z": "Moto Z", "samsung galaxy a8+": "Samsung Galaxy A8+", "sony xperia xzs": "Sony Xperia XZs", "3t": "OnePlus 3T", "lg g4": "LG G4", "huawei nexus 6p": "Huawei Nexus 6P", "u11 life": "HTC U11 Life", "v10": "LG V10", "galaxy s8": "Samsung Galaxy S8", "vivo": "BLU VIVO 8L", "xiaomi mi a1": "Xiaomi Mi A1", "2 xl": "Google Pixel 2 XL", "edge": "Samsung Galaxy S7 Edge", "u11+": "HTC U11+", "google pixel 2": "Google Pixel 2", "g4 plus": "Moto G4 Plus", "honor 7x": "Honor 7X", "v9": "ZTE Blade V9", "xzs": "Sony Xperia XZs", "honor view": "Honor View 10", "maze alpha": "Maze Alpha", "view 10": "Honor View 10", "g6": "LG G6", "lg v20": "LG V20", "oneplus 3t": "OnePlus 3T", "samsung galaxy s6": "Samsung Galaxy S6", "galaxy s7 edge": "Samsung Galaxy S7 Edge", "pixel": "Google Pixel", "mi a1": "Xiaomi Mi A1", "honor 9 lite": "Honor 9 Lite"}

MAX_FRAGMENT_SIZE=0 # so that we don't have to search till the end of the string
for frag, index in fragmentLookup.items():
    if frag.count(" ") + 1 > MAX_FRAGMENT_SIZE:
        MAX_FRAGMENT_SIZE = frag.count(" ") + 1
        
rgx = re.compile("[\w+]+") # include + also

def getBrands(text):
    brandsFound = set()
    words = set(rgx.findall(text))
    for brand in brands:
        if brand.lower() in words:
            brandsFound.add(brand)
    return brandsFound

def getModels(text):
    modelsFound = set()
    fragmentsFound = set()
    fragmentsAccepted = {}
    
    words = rgx.findall(text)
    for i in range(0, len(words)):
        if words[i] not in allModelTokens:
            continue
        for j in range(i+1, min(i+1+MAX_FRAGMENT_SIZE, len(words)+1)):
            fragment = " ".join(words[i:j])
            if fragment in fragmentLookup:
                fragmentsFound.add(fragment)
    for fragment in sorted(fragmentsFound, key=len, reverse=True):
        # we iterate through the fragments we've found, to ensure that 'smaller' 
        # model names are not accidentally extracted (e.g. Pixel vs Pixel 2)
        found = False
        for frag, index in fragmentsAccepted.items():
            if fragment in frag and text.find(frag) + frag.find(fragment) == text.find(fragment):
                found = True
                break
        if not found:
            fragmentsAccepted[fragment] = text.find(fragment)
    for frag, index in fragmentsAccepted.items():
        modelsFound.add(fragmentLookup[frag])
    return modelsFound

def getEntities(text):
    lowered = text.lower()
    brandsFound = getBrands(lowered)
    modelsFound = getModels(lowered)
    for model in modelsFound:
        brandsFound.add(model.split(" ", 1)[0])
    return (brandsFound, modelsFound)

## Make Summary for each model

In [12]:
# each phone here represents a row in the dataframe
def makeSummary(phone):
    phoneBrand = phone[0]
    phoneName = phone[1]
    phoneURL = phoneName.lower().replace(" ", "-")
    
    if os.path.exists("results/gsm/"+phoneURL+"_categories.json"):
        return
    
    sentiments = {}
    
    categories = []
    # data to be put in categories
    postsCount = 0
    try:
        postsCount = len(phone[2].split(";;;;;")[1:]) # ignore "" in first index
    except:
        postsCount = 0
    
    startDate = None
    endDate = None
    
    competitorBrands = {}
    competitorModels = {}
    
    categorySentiments = []
    try:
        allReviews = phone[2].split(";;;;;")[1:] 
        for review in allReviews:
            tb = TextBlob(review)
            brandsFound, modelsFound = getEntities(review)

            for brand in brandsFound:
                if brand != phoneBrand:
                    if brand in competitorBrands:
                        competitorBrands[brand] += 1
                    else:
                        competitorBrands[brand] = 1

            for model in modelsFound:
                if model != phoneName:
                    if model in competitorModels:
                        competitorModels[model] += 1
                    else:
                        competitorModels[model] = 1

            categorySentiments.append(tb.sentiment.polarity)
    except:
        pass
    
    catSentiments = None
    try:
        if len(categorySentiments) > 0:
            catSentiments = list(np.percentile(categorySentiments, [25, 50, 75]))
    except:
        pass
    
    categories.append({ 
        "num_posts": postsCount, 
        "sentiments": catSentiments, 
        "competitor_brands": competitorBrands, 
        "competitor_models": competitorModels
    })
    
    try:
        with open("results/gsm/"+phoneURL+"_categories.json", "w") as file:
            json.dump({"categories": categories}, file)
    except OSError:
        pass
    
    """for time in phone[3]:
        timestamp = datetime.strptime(solve(post['time']), "%d %B %Y, %I:%M %p").date()
        if timestamp in postCount:
            postCount[timestamp] += 1
            sentiments[timestamp].append(tb.sentiment.polarity)
        else:
            postCount[timestamp] = 1
            sentiments[timestamp] = [tb.sentiment.polarity]

        if startDate is None or timestamp < startDate:
            startDate = timestamp
        if endDate is None or timestamp > endDate:
            endDate = timestamp"""

In [13]:
for data in range(len(all_phones_df)):
    makeSummary(all_phones_df.iloc[data])

## Conversion of dataframes to phone specific dataframes
This section for testing.

In [153]:
import ast

test = samsung_df.iloc[0]
comments = test[2].split(";;;;;")[1:]
len(test[2].split(";;;;;")[1:])

date = []
print(type(test[3]))
x = test[3].strip('').strip('[]')
dt = re.
#datetime_object = datetime.strptime(x[0], '%b %d %Y %I:%M%p')
#print(datetime_object)
print(x)
print(type(x))

<class 'str'>
datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24), datetime.date(2018, 3, 24)
<class 'str'>


In [21]:
def calculate_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

def calculate_sentiment_for_all_phones(frame):
    total_sentiment = []
    
    # To account for first string which is empty due to splitting by ';;;;;'
    total_count = len(frame) - 1
    for model in range(len(frame)):
        total_current_phone_sentiment = 0
        average_current_phone_sentiment = 0
        length_current_phone_sentiment = 0
        split_model = frame['phone_reviews'][model].split(";;;;;")
        for sent in split_model:
            total_current_phone_sentiment += calculate_sentiment(sent)
            length_current_phone_sentiment += 1
        average_current_phone_sentiment = total_current_phone_sentiment/length_current_phone_sentiment
        total_sentiment.append(average_current_phone_sentiment)
    return total_sentiment

In [35]:
# Dataframe that contains only samsung phones
samsung_df = phone_df1.append([phone_df2.head(20)]).reset_index()
samsung_df.drop('index', axis=1, inplace=True)

In [66]:
# Dataframe that contains only apple phones
apple_df = pd.DataFrame(columns=['phone_brand', 'phone_model', 'phone_reviews', 'phone_reviews_dates'])

for phone in range(len(phone_df2)):
    if (phone_df2.iloc[phone]['phone_brand'] == 'Apple'):
        new_df = pd.DataFrame([[phone_df2.iloc[phone]['phone_brand'],phone_df2.iloc[phone]['phone_model'],
                              phone_df2.iloc[phone]['phone_reviews'],phone_df2.iloc[phone]['phone_reviews_dates']]],
                             columns=['phone_brand', 'phone_model', 'phone_reviews', 'phone_reviews_dates'])
        apple_df = apple_df.append(new_df)
apple_df = apple_df.reset_index()
apple_df.drop('index', axis=1, inplace=True)